<a href="https://colab.research.google.com/github/KakoAdam/NLP_GYAK_53/blob/main/vimiac22_gyak53.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ismerkedés a szóbeágyazásokkal (bmevimiac22 kurzus 5.3. gyakorlat)  
Ebben a gyakorlatban a szóbeágyazások elemi használatával ismerkedünk meg, kicsit belenézve a modellek belsejébe is.  
Későbbi gyakorlatokban látni fog ennél egyszerűbb megoldásokat is.

Letöltünk egy szóbeágyazás-modellt (Stanford GloVe [link text](https://github.com/stanfordnlp/GloVe)).  
Eltarthat egy ideig. Amíg fut, addig átnézheti a következő feladatokat.  
*Ha GloVe angol modellel megoldotta a feladatokat, akkor megpróbálkozhat más modellekkel is. Ennek részleteit a feladatsor végén találja.*

In [ ]:
!wget https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip

Kicsomagoljuk.

In [ ]:
!unzip glove.6B.zip

A fájlok nevében az első szám a tanításra használt korpusz méretét (tokenekben), míg a második a szóbeágyazás-vektor nagyságát írja le.

A modellfájlok soronként egy szót és annak vektorreprezentációját tartalmazzák.  
Nézze meg a king (király) szó reprezentációját a legkisebb modellben!  
Számolja meg, hány szót tartalmaz a legnagyobb modell!  
A feladatokat a `grep` és a `wc` Unix paranccsokkal oldja meg!  
(A Unix rendszerprogramjai közül számos eszköz jól használható szövegfeldolgozási feladatokban, pl.: grep, wc, sed, awk, perl.)

In [ ]:
!grep
!wc

Kiválasztunk egy modellt.  
Kisebb (akár 50d) modellel is lehet kísérletezni, de az eredmények egyes esetekben nem lesznek annyira látványosak. (Azért rosszak sem.)

In [ ]:
modelfile = 'glove.6B.300d.txt'

## A modell betöltése  
Egy olyan Python szótárat alkotunk, amelynek kulcsai a szavak, értékei pedig a szóbeágyazás-vektorok.  
A [numpy](https://numpy.org/) egy matematikai programcsomag, ami pár dolgot leegyszerűsít számunkra a későbbiekben.  
A modell betöltése percekig is tarthat. A haladást jelzi a `tqdm`, a sorok számát fentebb számoltuk meg a `wc` paranccsal.  
Kisebb (akár 50d) modellel is lehet kísérletezni, de az eredmények egyes esetekben nem lesznek annyira látványosak. (Azért rosszak sem.)  
Ha másik modellt választ, akkor a szavak összes számát (a `total` argumentumot) is ki kell javítani.

In [ ]:
import numpy as np
import os
from tqdm.notebook import trange, tqdm

with open(modelfile, "r") as f:
  embeddings = {l.split()[0]: np.array(l.split()[1:]).astype('float') for l in tqdm(f, total=400001)}

Kiírjuk a king szóhoz tartozó vektor első pár elemét.

In [ ]:
print(embeddings['king'][:6])

Hasonlítsuk össze néhány szópár vektorát!  
Az [euklideszi távolságmértéket](https://hu.wikipedia.org/wiki/T%C3%A1vols%C3%A1g#Az_euklideszi_t%C3%A9rben) (a Numpy csomag `linalg.norm()` függvényét) fogjuk használni.

In [ ]:
dist_kq = np.linalg.norm(embeddings['king'] - embeddings['queen'])
print(dist_kq)
dist_kc = np.linalg.norm(embeddings['king'] - embeddings['car'])
print(dist_kc)
dist_km = np.linalg.norm(embeddings['king'] - embeddings['man'])
print(dist_km)

Megfigyelhetjük, hogy a király közelebb áll a királynőhöz, mint az autóhoz vagy a férfihoz.  
Magyarázza el, mi lehet ennek az oka:  
TODO

Hasonlítsuk össze szópárok távolságát!

In [ ]:
dist_qw = np.linalg.norm(embeddings['queen'] - embeddings['woman'])
print(f"King - man = {dist_km:.3}\tvs.\tQueen - woman = {dist_qw:.3}")

A king és a férfi távolsága nagyon hasonlít a királynő és a nő távolságához.

Kísérletezzen további szópárokkal (pl. jelzők fokozásával, országokkal és fővárosaikkal stb.)!  
Megjegyzés: a szótárban minden szó kisbetűs.

In [ ]:
dist1 = np.linalg.norm(embeddings['madrid'] - embeddings['spain'])
dist2 = np.linalg.norm(embeddings['rome'] - embeddings['italy'])
print(f"{dist1:.3}\tvs.\t{dist2:.3}")
TODO

Próbáljon egy olyan programot írni, ami meghatározza X értékét az alább mondatban:  

> small is to smaller as big to X  

A program az első két szó mintáját követve meghatározza a harmadik szó párját, azaz minimalizálja a két szópár közötti különbségek eltérését.

In [ ]:
# a nyelvi modell (embeddings) globális változóban elérhető
def embed_associate(token1, token2, token3):
  basedist = np.linalg.norm(embeddings[token1] - embeddings[token2])
  min = 10
  res = 'No close match'
  for tok, emb in tqdm(embeddings.items()):
    TODO
  return res

print(embed_associate('small', 'smaller', 'big'))

Ha sikerült, kísérletezzen más példákkal is, és írja le a tapasztalatait!

In [ ]:
print(embed_associate('dog', 'puppy', 'cat'))
TODO

Tapasztalatok:  
TODO

## Szóbeágyazások vizualizációja
A beágyazásvektorok 300 dimenziós térben vannak. A vizualizációhoz 2D térbe kell ezeket transzformálnunk.  
A főkomponens-analízis ([Principal component analysis](https://scikit-learn.org/stable/modules/decomposition.html#pca), PCA) pont erre való, amely jelentős információvesztés nélkül igyekszik csökkenteni a dimenziószámot a megadott értékűre. (A módszer megismerése mellőzhető.)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
pca = PCA(n_components=2, whiten=True)

def embed_plot(tokens, color):
  # Lekérdezzük a tokenekhez (szavakhoz) tartozó vektorokat
  train_X = [embeddings[t] for t in tokens]

  # Illesztünk rá egy PCA modellt és transzformáljuk 2D térbe
  em2d = pca.fit_transform(train_X)

  # Kirajzoljuk az adatokat
  plt.scatter(em2d[:,0], em2d[:,1], c=color)

  # Felcímkézzük
  for label, x, y in zip(tokens, em2d[:,0], em2d[:,1]):
    plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')

plt.grid()
plt.rcParams['figure.figsize'] = (8, 6)
embed_plot(['dog', 'cat', 'dogs', 'cats', 'pet', 'puppy', 'car', 'king', 'queen', 'man', 'woman'], 'blue')
plt.show()


Szépen megfigyelhetők a cat - cats vs. dog - dogs párok távolságai, illetve a king, queen és a man, woman szavak elhelyezkedései.  
Kísérletezzen más szavakkal is!

In [ ]:
embed_plot(['big', 'bigger', 'small', 'smaller'], 'blue')
embed_plot(['budapest', 'hungary', 'rome', 'italy'], 'red')
plt.grid()
plt.rcParams['figure.figsize'] = (8, 6)
plt.show()


Ügyesebb eszközzel 3D modellt is készíthetünk.  
Kísérletezzen egy kicsit a [Tensorflow Embedding Projectorral](https://projector.tensorflow.org/)!

## Kísérletezés más modellekkel  
A fenti feladatsort megoldhatja más (pl. magyar nyelvű) szóbeágyazás-modell felhasználásával.  
(Ha változik a nyelv, akkor a fentebb használt szavakat is cserélnie kell.)  
Ne válasszon túl nagy modellt, mert előfordulhat, hogy a Google Colab erőforrásai nem lesznek elegendők a használatához!  
A letöltés általában elég lassú, néha a fájlműveletek is.


Letöltjük a modellt, pl. a [FastText](https://fasttext.cc/docs/en/pretrained-vectors.html)et.

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec

A FastText modell (főleg a magyar) a gyakorlat során igényel egy kis adattisztítást.  
Kiszűrjük azokat a sorokat, amelyekben a szó nem betűvel kezdődik és az első számíg (vagy - jelig) nem megfelelő karakterek vannak.  
Ez is eltart egy ideig...

In [ ]:
!grep "^[a-zéáíóöőüú][a-zéáíóöőüú\-_]* [-[:digit:]]" wiki.en.vec > wiki.en.cleaned.vec
!wc -l wiki.en.*.vec

A modellfájl beállítása után visszaléphet a *modell betöltése* lépésre.  
Ha nem angol nyelvű modellt választott, akkor cserélnie kell majd a szavakat a példákban.

In [ ]:
modelfile = 'wiki.en.cleaned.vec'